# Segmenting and Clustering Neighborhoods in Toronto

## Part 1 - Create Notebook
Done, here is my notebook I created!

## Part 2 - Create Dataframe from Wikipedia Page Containing Postal Codes in Canada Starting with M

In [72]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json
from pandas.io.json import json_normalize
import numpy as np

In [73]:
# Get to webpage and extract source code using bs4 module
headers = requests.utils.default_headers()
headers.update({ 'User-Agent': 'Chrome/6.0.472.63 Sfari/534.3a'})

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
req = requests.get(url, headers)
soup = BeautifulSoup(req.content, 'html.parser')

In [74]:
# Identify table to scrap data from 
tables = soup.find('table', {'class':'wikitable sortable'})
table_rows = tables.find_all('tr')

# Create empty list to input table values into from for loop below
data = []

# Using for loop to pull text from each cell and add to list
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

# Convert list to dataframe and remove row containing null 
df_tdot = pd.DataFrame(data, columns = ['Postal Code', 'Borough', 'Neighborhood'])
df_tdot.dropna(inplace=True)
df_tdot.head()

,Postal Code,Borough,Neighborhood
1,M1A,Not assigned,
2,M2A,Not assigned,
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Regent Park / Harbourfront


## Part 3 - Restructure Dataframe

In [75]:
# Ignore rows with Borough columnn containing 'Not assigned' by modifying existing dataframe to only include rows where Borough column does not match 'Not assigned'
df_tdot = df_tdot[df_tdot['Borough'] != 'Not assigned']

# Groupby function used to group Postal Code column so more than one Neighborhood can exist for a postal code row
df_tdot.groupby(by = ['Postal Code'], axis = 0)

# Reset index as it is missing values after the groupby function is applied
df_tdot.reset_index(drop = True, inplace = True)

# The groupby function defaulted seperating multiple Neighborhoods in a cell with '/'.  Used replace function to replace '/' with a comma.
df_tdot['Neighborhood'] = df_tdot['Neighborhood'].str.replace('/',',')

# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.  Note that no neighborhoods in scraped table has contains 'Not assigned'.
mask = df_tdot['Neighborhood'] == 'Not assigned'
df_tdot.loc[mask, 'Neighborhood'] = df_tdot.loc[mask, 'Borough']

df_tdot.head()


,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


In [76]:
# Show shape of dataframe after dataframe is pre-processed
df_tdot.shape

(103, 3)

## Imported Coordinates into Dataframe from CSV file

In [77]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Merge Dataframes so Coordinates are Added

In [78]:
df_tdot_coords = pd.merge(df_tdot, df_coords, on = 'Postal Code', how = 'inner')
df_tdot_coords.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494


# Part 4 - Visualize/Cluster/Examine Neighborhoods

In [110]:
#!conda install -c conda-forge folium=0.5.0 --yes
import folium

## Find Coordinates of Toronto to Centralize Map

In [80]:
from geopy.geocoders import Nominatim
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="tdot_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


## Create Map to Visualize Boroughs and Neighborhoods

In [81]:
map_tdot = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, borough, neighborhood in zip(df_tdot_coords['Latitude'], df_tdot_coords['Longitude'], df_tdot_coords['Borough'], df_tdot_coords['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tdot)  

map_tdot

## Narrow Down Data to Downtown Toronto Borough Only Which We Will Explore Further

In [82]:
downtown_tdot_data = df_tdot_coords[df_tdot_coords['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown_tdot_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


## Define Foursquare Credentials and Version

In [83]:
CLIENT_ID = 'X2NTRDTQ4DLD0CQYX5IV340HD3DC5C2W1U4G2TRHTIWR4XA0' # your Foursquare ID
CLIENT_SECRET = 'L05JQ4HPYNVRZV13A32ZMC1GBSXYXZMJC0IRZAICQDTTZEEP' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: X2NTRDTQ4DLD0CQYX5IV340HD3DC5C2W1U4G2TRHTIWR4XA0
CLIENT_SECRET:L05JQ4HPYNVRZV13A32ZMC1GBSXYXZMJC0IRZAICQDTTZEEP


## Mapping Out Neighbourhoods in Downtown Toronto

In [84]:
# create map of Manhattan using latitude and longitude values
map_downtown_tdot = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, label in zip(downtown_tdot_data['Latitude'], downtown_tdot_data['Longitude'], downtown_tdot_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown_tdot)  
    
map_downtown_tdot

## Exploring First Neighborhood in Downtown Toronto Dataframe

In [85]:
neighborhood_latitude = downtown_tdot_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = downtown_tdot_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = downtown_tdot_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park , Harbourfront are 43.6542599, -79.3606359.


## Explore Top 100 Venues with a 500 meter radius around Regent Park and Harbourfront

In [86]:
# type your answer here
limit = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    limit)

print(url)

https://api.foursquare.com/v2/venues/explore?&client_id=X2NTRDTQ4DLD0CQYX5IV340HD3DC5C2W1U4G2TRHTIWR4XA0&client_secret=L05JQ4HPYNVRZV13A32ZMC1GBSXYXZMJC0IRZAICQDTTZEEP&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=100


In [87]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ea5d99683525f001bd44227'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 48,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
 

In [88]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [89]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149
3,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
4,Body Blitz Spa East,Spa,43.654735,-79.359874


In [90]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

48 venues were returned by Foursquare.


## Explore Neighborhoods in Downtown Toronto

### Creating a function to repeat the same process as above to all of Toronto's neighborhoods

In [91]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [92]:
downtown_toronto_venues = getNearbyVenues(names=downtown_tdot_data['Neighborhood'],
                                   latitudes=downtown_tdot_data['Latitude'],
                                   longitudes=downtown_tdot_data['Longitude']
                                  )

Regent Park , Harbourfront
Queen's Park , Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond , Adelaide , King
Harbourfront East , Union Station , Toronto Islands
Toronto Dominion Centre , Design Exchange
Commerce Court , Victoria Hotel
University of Toronto , Harbord
Kensington Market , Chinatown , Grange Park
CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport
Rosedale
Stn A PO Boxes
St. James Town , Cabbagetown
First Canadian Place , Underground city
Church and Wellesley


In [93]:
print(downtown_toronto_venues.shape)
downtown_toronto_venues.head()


(1225, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park , Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park , Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park , Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,"Regent Park , Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,"Regent Park , Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


### Check how many venues were returned for each neighborhood

In [94]:
downtown_toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,58,58,58,58,58,58
"CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport",14,14,14,14,14,14
Central Bay Street,62,62,62,62,62,62
Christie,18,18,18,18,18,18
Church and Wellesley,74,74,74,74,74,74
"Commerce Court , Victoria Hotel",100,100,100,100,100,100
"First Canadian Place , Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront East , Union Station , Toronto Islands",100,100,100,100,100,100


### Check how many unique categories from the venues returned above

In [95]:
print('There are {} uniques categories.'.format(len(downtown_toronto_venues['Venue Category'].unique())))

There are 205 uniques categories.


## Analyze Each Neighborhood in Downtown Toronto

In [96]:
# one hot encoding
tdot_onehot = pd.get_dummies(downtown_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tdot_onehot['Neighborhood'] = downtown_toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [tdot_onehot.columns[-1]] + list(tdot_onehot.columns[:-1])
tdot_onehot = tdot_onehot[fixed_columns]

print(tdot_onehot.shape)
tdot_onehot.head()

(1225, 205)


,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Group rows by neighborhood by taking the mean of the frequency of occurrence of each category

In [97]:
tdot_grouped = tdot_onehot.groupby('Neighborhood').mean().reset_index()
print(tdot_grouped.shape)
tdot_grouped.head()

(19, 205)


,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.017241,0.0,0.0,0.0,0.0,0.0
1,"CN Tower , King and Spadina , Railway Lands , ...",0.000000,0.071429,0.071429,0.142857,0.142857,0.142857,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,Central Bay Street,0.016129,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.016129,0.0,0.0,0.0,0.0,0.0
3,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,Church and Wellesley,0.027027,0.000000,0.000000,0.000000,0.000000,0.000000,0.013514,0.0,0.0,...,0.013514,0.013514,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


### Print each neighborhood along with the top 5 most common venues

In [98]:
num_top_venues = 5

for hood in tdot_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = tdot_grouped[tdot_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2  Seafood Restaurant  0.03
3  Italian Restaurant  0.03
4          Restaurant  0.03


----CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport----
                venue  freq
0      Airport Lounge  0.14
1     Airport Service  0.14
2    Airport Terminal  0.14
3       Boat or Ferry  0.07
4  Airport Food Court  0.07


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.21
1                Café  0.08
2  Italian Restaurant  0.06
3      Sandwich Place  0.05
4         Salad Place  0.03


----Christie----
           venue  freq
0  Grocery Store  0.22
1           Café  0.17
2           Park  0.11
3     Baby Store  0.06
4      Nightclub  0.06


----Church and Wellesley----
                 venue  freq
0     Sushi Restaurant  0.08
1          Coffee Shop  0.07
2  Japanese Restaurant  0.07
3           Restaura

### Create a function to sort the venues in descending order to put into a dataframe for top 10 venues in each neighborhood

In [99]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [100]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tdot_grouped['Neighborhood']

for ind in np.arange(tdot_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tdot_grouped.iloc[ind, :], num_top_venues)

print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted.head()

(19, 11)


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Café,Beer Bar,Farmers Market,Bakery,Restaurant,Cheese Shop,Italian Restaurant,Seafood Restaurant
1,"CN Tower , King and Spadina , Railway Lands , ...",Airport Lounge,Airport Service,Airport Terminal,Plane,Harbor / Marina,Sculpture Garden,Boat or Ferry,Rental Car Location,Boutique,Airport
2,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Ice Cream Shop,Salad Place,Burger Joint,Bubble Tea Shop,Fried Chicken Joint,Gastropub
3,Christie,Grocery Store,Café,Park,Candy Store,Nightclub,Coffee Shop,Restaurant,Gas Station,Italian Restaurant,Athletics & Sports
4,Church and Wellesley,Sushi Restaurant,Coffee Shop,Japanese Restaurant,Restaurant,Gay Bar,Hotel,Pub,Men's Store,Mediterranean Restaurant,Yoga Studio


## Cluster Neighborhoods - 5 Clusters

In [101]:
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [102]:
# set number of clusters
kclusters = 5

tdot_grouped_clustering = tdot_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tdot_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 3, 4, 2, 0, 0, 0, 0, 0, 0], dtype=int32)

### Create new dataframe that includes cluster and top 10 venues in neighborhood

In [103]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tdot_merged = downtown_tdot_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tdot_merged = tdot_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

print(tdot_merged.shape)
tdot_merged.head() 

(19, 16)


,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636,0,Coffee Shop,Pub,Park,Bakery,Breakfast Spot,Theater,Café,Restaurant,Playground,Electronics Store
1,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494,4,Coffee Shop,Sushi Restaurant,Diner,Gym,Mexican Restaurant,Juice Bar,Japanese Restaurant,Italian Restaurant,Hobby Shop,Fried Chicken Joint
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Café,Restaurant,Japanese Restaurant,Bubble Tea Shop,Cosmetics Shop,Middle Eastern Restaurant,Ramen Restaurant,Lingerie Store
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Gastropub,Cocktail Bar,Italian Restaurant,American Restaurant,Gym,Farmers Market,Lingerie Store,Creperie
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Café,Beer Bar,Farmers Market,Bakery,Restaurant,Cheese Shop,Italian Restaurant,Seafood Restaurant


## Visualize the Clusters - Create 5 Clusters

In [104]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tdot_merged['Latitude'], tdot_merged['Longitude'], tdot_merged['Neighborhood'], tdot_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters - Top 10 Most Common Venues for Neighborhood Clusters

### Cluster 1

In [105]:
tdot_merged.loc[tdot_merged['Cluster Labels'] == 0, tdot_merged.columns[[2] + list(range(6, tdot_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park , Harbourfront",Coffee Shop,Pub,Park,Bakery,Breakfast Spot,Theater,Café,Restaurant,Playground,Electronics Store
2,"Garden District, Ryerson",Clothing Store,Coffee Shop,Café,Restaurant,Japanese Restaurant,Bubble Tea Shop,Cosmetics Shop,Middle Eastern Restaurant,Ramen Restaurant,Lingerie Store
3,St. James Town,Coffee Shop,Café,Gastropub,Cocktail Bar,Italian Restaurant,American Restaurant,Gym,Farmers Market,Lingerie Store,Creperie
4,Berczy Park,Coffee Shop,Cocktail Bar,Café,Beer Bar,Farmers Market,Bakery,Restaurant,Cheese Shop,Italian Restaurant,Seafood Restaurant
7,"Richmond , Adelaide , King",Coffee Shop,Café,Restaurant,Gym,Thai Restaurant,Hotel,Clothing Store,Deli / Bodega,American Restaurant,Bookstore
8,"Harbourfront East , Union Station , Toronto Is...",Coffee Shop,Aquarium,Hotel,Café,Fried Chicken Joint,Restaurant,Sporting Goods Shop,Italian Restaurant,Brewery,Scenic Lookout
9,"Toronto Dominion Centre , Design Exchange",Coffee Shop,Hotel,Café,Restaurant,Japanese Restaurant,American Restaurant,Salad Place,Deli / Bodega,Seafood Restaurant,Sporting Goods Shop
10,"Commerce Court , Victoria Hotel",Coffee Shop,Restaurant,Café,Hotel,American Restaurant,Gym,Italian Restaurant,Seafood Restaurant,Deli / Bodega,Japanese Restaurant
11,"University of Toronto , Harbord",Café,Bookstore,Japanese Restaurant,Restaurant,Bar,Italian Restaurant,Bakery,Beer Bar,Beer Store,Sandwich Place
12,"Kensington Market , Chinatown , Grange Park",Café,Coffee Shop,Vietnamese Restaurant,Mexican Restaurant,Bakery,Dessert Shop,Bar,Vegetarian / Vegan Restaurant,Gaming Cafe,Grocery Store


### Cluster 2

In [106]:
tdot_merged.loc[tdot_merged['Cluster Labels'] == 1, tdot_merged.columns[[2] + list(range(6, tdot_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Rosedale,Park,Trail,Playground,Cupcake Shop,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner


### Cluster 3

In [107]:
tdot_merged.loc[tdot_merged['Cluster Labels'] == 2, tdot_merged.columns[[2] + list(range(6, tdot_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Christie,Grocery Store,Café,Park,Candy Store,Nightclub,Coffee Shop,Restaurant,Gas Station,Italian Restaurant,Athletics & Sports


### Cluster 4

In [108]:
tdot_merged.loc[tdot_merged['Cluster Labels'] == 3, tdot_merged.columns[[2] + list(range(6, tdot_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,"CN Tower , King and Spadina , Railway Lands , ...",Airport Lounge,Airport Service,Airport Terminal,Plane,Harbor / Marina,Sculpture Garden,Boat or Ferry,Rental Car Location,Boutique,Airport


### Cluster 5

In [109]:
tdot_merged.loc[tdot_merged['Cluster Labels'] == 4, tdot_merged.columns[[2] + list(range(6, tdot_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Queen's Park , Ontario Provincial Government",Coffee Shop,Sushi Restaurant,Diner,Gym,Mexican Restaurant,Juice Bar,Japanese Restaurant,Italian Restaurant,Hobby Shop,Fried Chicken Joint
5,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Ice Cream Shop,Salad Place,Burger Joint,Bubble Tea Shop,Fried Chicken Joint,Gastropub
